# Декораторы

In [1]:
def foo(x):
    print(x + 1)


def decorator(foo):
    def _dec(*args, **kwargs):
        print("inside decorator")
        res = foo(*args, **kwargs)
        return res
    
    return _dec

In [2]:
foo(1)

2


In [3]:
foo = decorator(foo)
foo(1)

inside decorator
2


### В Python есть специальный синтаксис, который позволяет переместить модификацию функции ближе к моменту объявления функции:

In [4]:
@decorator
def foo(x):
    """
    foo description
    """
    print(x + 1)

In [5]:
foo(1)

inside decorator
2


### Есть небольшая проблема:

In [9]:
# ?????
foo.__name__, foo.__doc__

('foo', '\n    foo description\n    ')

#### Еще "пропадает" ```__module__```
#### Что делать?

In [7]:
def decorator(foo):
    def _dec(*args, **kwargs):
        res = foo(*args, **kwargs)
        return res
    # ? SHTA ?
    _dec.__name__ = foo.__name__
    _dec.__doc__ = foo.__doc__
    _dec.__module__ = foo.__module__
    
    return _dec


@decorator
def foo(x):
    """
    foo description
    """
    print(x + 1)

In [8]:
foo.__name__, foo.__doc__

('foo', '\n    foo description\n    ')

#### Но это сложно, можно проще.
#### Воспользуемся декоратором ```wraps``` из пакета ```functools``` <i>(подробнее про этот пакет чуть позже)</i>

In [10]:
from functools import wraps


def decorator(foo):
    @wraps(foo)
    def _dec(*args, **kwargs):
        res = foo(*args, **kwargs)
        return res
    
    return _dec


@decorator
def foo(x):
    """
    foo description
    """
    print(x + 1)

In [11]:
foo.__name__, foo.__doc__

('foo', '\n    foo description\n    ')

### Декораторов может быть несколько:

In [12]:
def dec1(func):
    @wraps(func)
    def _dec(*args, **kwargs):
        print("inside dec1")
        res = func(*args, **kwargs)
        return res
        
    return _dec


def dec2(func):
    @wraps(func)
    def _dec(*args, **kwargs):
        print("inside dec2")
        res = func(*args, **kwargs)
        return res
        
    return _dec

In [15]:
@dec2
@dec1
# output dec2 -> dec1
def func(arg1, arg2):
    pass

In [16]:
func(1, 2)

inside dec2
inside dec1


#### Это же самое, что:

In [25]:
def func(arg1, arg2):
    pass

In [26]:
func = dec2(dec1(func))

In [27]:
func(1, 2)

inside dec2
inside dec1


### Вопрос [?]

In [34]:
flag = False


def decorator(func):
    @wraps(func)
    def _dec(*args, **kwargs):
        print("inside _dec")
        res = func(*args, **kwargs)
        return res
    
    return func if flag else _dec


# @decorator
def func():
    pass

func = decorator(func)

In [35]:
func()

inside _dec


#### Что будет?

In [36]:
flag = True
func = decorator(func)
func()

inside _dec


### В декоратор можно передавать аргументы:

In [45]:
# натыкать принтов
def decorator_with_args(dec_argument):
    def _decorator(func):
        @wraps(func)
        def _dec(*args, **kwargs):
            print(f"inside decorator; {dec_argument}")
            res = func(*args, **kwargs)
            return res
        
        return _dec
    return _decorator

In [46]:
@decorator_with_args("hop hey lala ley")
def func(x):
    pass

In [47]:
func(None)

inside decorator; hop hey lala ley


#### Это то же самое, что:

In [52]:
def decorator_with_args(dec_argument):
    def _decorator(func):
        @wraps(func)
        def _dec(*args, **kwargs):
            print(f"inside decorator; {dec_argument}")
            res = func(*args, **kwargs)
            return res
        
        return _dec
    return _decorator

In [53]:
def func(x):
    pass

In [54]:
decorator = decorator_with_args("hop hey lala ley")  # получим тут декоратор
func = decorator(func)  # получим модифицированную функцию

In [55]:
func(None)

inside decorator; hop hey lala ley


#### А если мы хотим опциональные аргументы?

In [59]:
# после звездочки нельзя передавать аргумент без имени

def decorator_with_optional_arguments(func=None, *, dec_argument="default"):
    if func is None:
        return lambda func: decorator_with_optional_arguments(func, dec_argument=dec_argument)
    @wraps(func)
    def _dec(*args, **kwargs):
        print(f"inside decorator; {dec_argument}")
        res = func(*args, **kwargs)
        return res
    
    return _dec

In [60]:
@decorator_with_optional_arguments(dec_argument="Life is beatiful")
def func1():
    pass


func1()

inside decorator; Life is beatiful


In [61]:
@decorator_with_optional_arguments
def func2():
    pass


func2()

inside decorator; default


### Несколько примеров полезных декораторов

#### Посчитать, сколько раз выполнялась функция:

In [62]:
def profiled(func):
    @wraps(func)
    def inner(*args, **kwargs):
        inner.ncalls += 1
        return func(*args, **kwargs)
    inner.ncalls = 0
    return inner

In [63]:
@profiled
def f():
    pass


for i in range(1000):
    f()
    
    
print(f.ncalls)

1000


#### Декоратор, для вызова декорируемой функции только 1 раз

In [68]:
# TODO
def once(func):
    @wraps(func)
    def _dec(*args, **kwargs):
        if _dec.flag:
            raise Exception
        _dec.flag = True
        return func(*args, **kwargs)
    _dec.flag = False
    return _dec

@once
def f():
    pass
f()

In [69]:
f()

Exception: 

* [contextlib.contextmanager](https://docs.python.org/3.5/library/contextlib.html#contextlib.contextmanager) -- Декоратор для обертки функции, чтобы использовать ее с with
* [functools.lru_cache](https://docs.python.org/3.5/library/functools.html#functools.lru_cache)
* почти все в [pycontracts](https://andreacensi.github.io/contracts/)
* @classmethod и @staticmethod
* @property
* @abstractmethod

...

# Области видимости

In [37]:
print(min)  # build-in

a = 0  # global
b = 1  # global


def g():
    a = -1  # enclosing
    b = -2  # enclosing
    
    def f():
        a = 1  # local
        b = 2  # local

        print("locals: ", locals())

        print("a in globals=", globals()["a"], sep="")
        print("b in globals=", globals()["b"], sep="")
        
    f()
    
    
g()

<built-in function min>
locals:  {'b': 2, 'a': 1}
a in globals=0
b in globals=1


### А что с присваиванием?

In [38]:
a = -1  # global


def f():
    try:
        a += 1  # local
    except UnboundLocalError:
        print("Catch UnboundLocalError")


f()

Catch UnboundLocalError


#### Можно ли как-то присвоить? Да!

In [39]:
a = -1  # global


def f():
    globals()["a"] += 1  # global


f()
print(a)

0


#### Или так:

In [42]:
a = -1  # global


def f():
    global a
    a += 1  # global
    print(a)


f()

0


#### Для ```enclosing```, соответственно, вот так вот:

In [43]:
def g():
    a = -1  # enclosing

    def f():
        nonlocal a
        a += 1  # local

    f()
    print(a)
    
    
g()

0


# Дескрипторы

### @property

In [70]:
# property - делает 

class MoneySaver:
    def __init__(self, exchange_rate):
        self.exchange_rate = exchange_rate
        self.copilka = 0
        
    @property
    def copilka(self):
        return self.copilka * self.exchange_rate
    
    @copilka.setter
    def copilka(self, x):
        self.copilka += x / self.exchange_rate

In [71]:
saver = MoneySaver(100)
saver.copilka = 10
print(saver.copilka)

RecursionError: maximum recursion depth exceeded while calling a Python object

In [72]:
class SomeBank:
    def __init__(self, start):
        self._rubles = start
    
    # хотим, чтобы rubles был как атрибут
    # __get__
    @property
    def rubles(self):
        return self._rubles
    # __set__ - поведение при присваивании элемента
    @rubles.setter
    def rubles(self, value):
        if value > 0:
            self._rubles = value
        else:
            raise Exception('You shall not pass!')

    @rubles.deleter
    def rubles(self, value):
        del self._rubles

In [73]:
interv = SomeBank(6000)

In [74]:
interv.rubles

6000

In [75]:
interv.rubles = 4242

In [76]:
interv.rubles = -32

Exception: You shall not pass!

### \_\_get__(), \_\_set__() и \_\_delete__(), 

In [91]:
class Descr:
    def __get__(self, instance, owner):
        print(instance, owner)
        
    def __set__(self, instance, value):
        print(instance, value)
        
    def __delete__(self, instance):
        print(instance)
        
    
class A:
    attr = Descr()

In [84]:
A().attr

<__main__.A object at 0x1050ede80> <class '__main__.A'>


In [85]:
A.attr

None <class '__main__.A'>


In [86]:
A().attr = 42

<__main__.A object at 0x1050edf98> 42


#### Что будет?

In [87]:
A.attr = 42

In [88]:
del A().attr

AttributeError: attr

In [89]:
del A.attr

### Хм

* `instance` -- экз. класса
* `attr` -- атрибут (который дескриптор)
* `descr = cls.__dict__["attr"]` -- сам дескриптор

In [92]:
instance = A()
descr = A.__dict__["attr"]

####  Тогда:

In [93]:
A.attr
descr.__get__(None, A)

None <class '__main__.A'>
None <class '__main__.A'>


In [94]:
instance.attr
descr.__get__(instance, A)

<__main__.A object at 0x1050f1a90> <class '__main__.A'>
<__main__.A object at 0x1050f1a90> <class '__main__.A'>


In [95]:
instance.attr = 42
descr.__set__(instance, 42)

<__main__.A object at 0x1050f1a90> 42
<__main__.A object at 0x1050f1a90> 42


In [96]:
del instance.attr
descr.__delete__(instance)

### Вопрос [?]

#### Где хранить данные дескриптору?

### Еще Вопрос [?]

#### Как вы думаете, как работают методы в Python?

`types.MethodType`

* http://stupidpythonideas.blogspot.com/2013/06/how-methods-work.html
* http://igorsobreira.com/2011/02/06/adding-methods-dynamically-in-python.html